In [1]:
from reader_utils.corpus_builder_utils import *
from note_utils.pitch_dictionary import PitchDictionary
from reader_utils.dataset_builder import DatasetBuilder
from model_utils.model_utils import *
import io

In [2]:
pd = PitchDictionary("dataset_objects/pitches_dict.txt")

vocab_size = pd.get_vocabulary_size()
window_size = 2
num_ns = 2
embedding_layer_name = "n2v_embedding"

In [3]:
db = DatasetBuilder("dataset_objects/full_corpus")

In [4]:
# takes around 24 mins for window_size = 2, num_ns = 2
dataset = db.build_word_to_vec_dataset(
        vocab_size, window_size, num_ns, skip_amount=1)

100%|██████████| 2200/2200 [14:35<00:00,  2.51it/s]


In [6]:
embedding_dim = 16

n2v = build_note_to_vec_model(
        vocab_size, embedding_dim, num_ns, embedding_layer_name)

In [7]:
# takes around 11 mins for embedding_dim = 32
epochs = 20
n2v.fit(dataset, epochs=epochs)

Epoch 1/20
3428/3428 [==============================] - 18s 5ms/step - loss: 0.1865 - accuracy: 0.9501
Epoch 2/20
3428/3428 [==============================] - 8s 2ms/step - loss: 0.1090 - accuracy: 0.9622
Epoch 3/20
3428/3428 [==============================] - 8s 2ms/step - loss: 0.1026 - accuracy: 0.9645
Epoch 4/20
3428/3428 [==============================] - 8s 2ms/step - loss: 0.0997 - accuracy: 0.9655
Epoch 5/20
3428/3428 [==============================] - 8s 2ms/step - loss: 0.0986 - accuracy: 0.9658
Epoch 6/20
3428/3428 [==============================] - 8s 2ms/step - loss: 0.0980 - accuracy: 0.9660
Epoch 7/20
3428/3428 [==============================] - 8s 2ms/step - loss: 0.0978 - accuracy: 0.9661
Epoch 8/20
3428/3428 [==============================] - 8s 2ms/step - loss: 0.0976 - accuracy: 0.9662
Epoch 9/20
3428/3428 [==============================] - 8s 2ms/step - loss: 0.0975 - accuracy: 0.9662
Epoch 10/20
3428/3428 [==============================] - 8s 2ms/step - loss: 0.09

In [8]:
vocab = pd.get_vocabulary()
weights = n2v.get_layer(embedding_layer_name).get_weights()[0]

In [9]:
file_suffix = "ws" + str(window_size) + "_ns" + str(num_ns) + \
        "_ed" + str(embedding_dim) + "_ep" + str(epochs)
vactors_file_name = "vectors_" + file_suffix + ".tsv"
metadata_file_name = "metadata_" + file_suffix + ".tsv"

vectors_file = io.open(vactors_file_name, 'w', encoding='utf-8')
metadata_file = io.open(metadata_file_name, 'w', encoding='utf-8')

for index, word in enumerate(vocab):
    if index in [0, 1]: # <unk> and es
        continue
    vec = weights[index]
    vectors_file.write('\t'.join([str(x) for x in vec]) + "\n")
    metadata_file.write(word + "\n")

vectors_file.close()
metadata_file.close()